In [1]:
# Download YOLOv5
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
# Install dependencies
%pip install -qr requirements.txt  # install dependencies
%cd ../

Cloning into 'yolov5'...
remote: Enumerating objects: 9827, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 9827 (delta 0), reused 2 (delta 0), pack-reused 9822
Receiving objects: 100% (9827/9827), 10.11 MiB | 18.96 MiB/s, done.
Resolving deltas: 100% (6829/6829), done.
/kaggle/working/yolov5
Note: you may need to restart the kernel to use updated packages.
/kaggle/working


In [2]:
import os
import gc
import cv2
import yaml
import torch
import random
import shutil
import warnings
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image, ImageDraw
from shutil import copyfile
from IPython.core.display import Video, display
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold

warnings.simplefilter('ignore')
pd.set_option("max_columns", 150)
pd.set_option('display.max_rows', 150)
# turn off W&B syncing if you don't need
os.environ['WANDB_MODE'] = 'offline'

### Configuration

In [3]:
CFG = {
    "seed"       : 42,
    "model"      : "yolov5m",
    "input_path" : '../input/nfl-health-and-safety-helmet-assignment/images/',
    "output_path": './nfl/',
    "batch_size" : 24,
    "epochs"     : 8,
    "size"       : 736,
    "fold_num"   : 4,
    "fold_break" : 2
}

CFG

{'seed': 42,
 'model': 'yolov5m',
 'input_path': '../input/nfl-health-and-safety-helmet-assignment/images/',
 'output_path': './nfl/',
 'batch_size': 24,
 'epochs': 8,
 'size': 736,
 'fold_num': 4,
 'fold_break': 2}

In [4]:
def seed_everything(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(CFG["seed"])

### Load data

In [5]:
# Load image level csv file
df_train = pd.read_csv('../input/nfl-health-and-safety-helmet-assignment/image_labels.csv')
print("Before")
print(df_train.label.value_counts())

df_train["right"]       = df_train.left + df_train.width
df_train["bottom"]      = df_train.top  + df_train.height
df_train["x_center"]    = df_train.left + (df_train.width  / 2).astype(int)
df_train["y_center"]    = df_train.top  + (df_train.height / 2).astype(int)
df_train["video_frame"] = df_train.image.apply(lambda x: "_".join(x.split("_")[:3]))
df_train["label"]       = df_train.label.apply(lambda x: x if x=="Helmet-Sideline" else "Helmet")
df_train["label_id"]    = df_train.label.apply(lambda x: 1 if x=="Helmet-Sideline" else 0)

print("-------------------------")
print("After")
print(df_train.label.value_counts())

print("-------------------------")
print(df_train.shape)
df_train.head()

Before
Helmet              129764
Helmet-Blurred       33544
Helmet-Sideline      15037
Helmet-Partial        8814
Helmet-Difficult      6577
Name: label, dtype: int64
-------------------------
After
Helmet             178699
Helmet-Sideline     15037
Name: label, dtype: int64
-------------------------
(193736, 12)


,image,label,left,width,top,height,right,bottom,x_center,y_center,video_frame,label_id
0,57503_000116_Endzone_frame443.jpg,Helmet,1099,16,456,15,1115,471,1107,463,57503_000116_Endzone,0
1,57503_000116_Endzone_frame443.jpg,Helmet,1117,15,478,16,1132,494,1124,486,57503_000116_Endzone,0
2,57503_000116_Endzone_frame443.jpg,Helmet,828,16,511,15,844,526,836,518,57503_000116_Endzone,0
3,57503_000116_Endzone_frame443.jpg,Helmet,746,16,519,16,762,535,754,527,57503_000116_Endzone,0
4,57503_000116_Endzone_frame443.jpg,Helmet,678,17,554,17,695,571,686,562,57503_000116_Endzone,0


In [6]:
kf = GroupKFold(n_splits=CFG["fold_num"])
for i, (tr_idx, va_idx) in enumerate(kf.split(df_train, None, df_train.video_frame)):
    df_train[f"fold{i}"] = 0
    df_train.loc[tr_idx, f"fold{i}"] = 1
    _sum = df_train[f"fold{i}"].sum()
    print(f"Fold {i}")
    print(f"Training data: {_sum}")
    print(df_train[df_train[f"fold{i}"]==1].label.value_counts())
    print(f"Validation data: {df_train.shape[0] - _sum}")
    print(df_train[df_train[f"fold{i}"]==0].label.value_counts())
    print("-------------------------")
    
df_train.head()

Fold 0
Training data: 145302
Helmet             134106
Helmet-Sideline     11196
Name: label, dtype: int64
Validation data: 48434
Helmet             44593
Helmet-Sideline     3841
Name: label, dtype: int64
-------------------------
Fold 1
Training data: 145302
Helmet             133942
Helmet-Sideline     11360
Name: label, dtype: int64
Validation data: 48434
Helmet             44757
Helmet-Sideline     3677
Name: label, dtype: int64
-------------------------
Fold 2
Training data: 145302
Helmet             133902
Helmet-Sideline     11400
Name: label, dtype: int64
Validation data: 48434
Helmet             44797
Helmet-Sideline     3637
Name: label, dtype: int64
-------------------------
Fold 3
Training data: 145302
Helmet             134147
Helmet-Sideline     11155
Name: label, dtype: int64
Validation data: 48434
Helmet             44552
Helmet-Sideline     3882
Name: label, dtype: int64
-------------------------


,image,label,left,width,top,height,right,bottom,x_center,y_center,video_frame,label_id,fold0,fold1,fold2,fold3
0,57503_000116_Endzone_frame443.jpg,Helmet,1099,16,456,15,1115,471,1107,463,57503_000116_Endzone,0,0,1,1,1
1,57503_000116_Endzone_frame443.jpg,Helmet,1117,15,478,16,1132,494,1124,486,57503_000116_Endzone,0,0,1,1,1
2,57503_000116_Endzone_frame443.jpg,Helmet,828,16,511,15,844,526,836,518,57503_000116_Endzone,0,0,1,1,1
3,57503_000116_Endzone_frame443.jpg,Helmet,746,16,519,16,762,535,754,527,57503_000116_Endzone,0,0,1,1,1
4,57503_000116_Endzone_frame443.jpg,Helmet,678,17,554,17,695,571,686,562,57503_000116_Endzone,0,0,1,1,1


### Make YOLO configuration files

In [7]:
def get_range(df, org_size):
    oh, ow = org_size[:2]
    ratio  = ow/oh
    x_min, x_max = df.left.min(),  df.right.max()
    y_min, y_max = df.top.min(),   df.bottom.max()
    w_max, h_max = df.width.max(), df.height.max()
    x_min = 0  if x_min < w_max else x_min - w_max
    x_max = ow if x_max > ow - w_max else x_max + w_max
    y_min = 0  if y_min < h_max else y_min - h_max
    y_max = oh if y_max > oh -h_max  else y_max + h_max
    
    cw = x_max - x_min
    ch = y_max - y_min
    rw = cw / ow
    rh = ch / oh 
    if rw / rh > 0:
        r = int((cw / ratio - ch) / 2)
        y_min = 0  if y_min - r < 0  else y_min - r
        y_max = oh if y_max + r > oh else y_max + r
    else:
        r = int((ch * ratio - cw) / 2)
        x_min = 0  if x_min - r < 0  else x_min - r
        x_max = ow if x_max + r > ow else x_max + r
    
    return x_min, x_max, y_min, y_max

def crop_image(df, input_path, output_path):
    org_size = (720, 1280)
    x_min, x_max, y_min, y_max = get_range(df, org_size)
    img     = cv2.imread(input_path)
    img_cut = img[y_min:y_max, x_min:x_max]
    cv2.imwrite(output_path, img_cut)
    return x_min, x_max, y_min, y_max
        
def create_file(df, fold, input_path, output_path):
    
    def make(_df, tr_val):
        for img_name in tqdm(_df.image.unique()):
            df_this_frame = _df[_df['image']==img_name]
            # Crop the side that there are no helmets
            x_min, x_max, y_min, y_max = crop_image(df_this_frame,
                                                    f'{input_path}{img_name}',
                                                    f'{output_path}images/{tr_val}/{fold}/{img_name}')
            with open(f'{output_path}labels/{tr_val}/{fold}/{img_name[:-4]}.txt', 'w+') as f:
                row = df_this_frame[['label_id','x_center','y_center','width','height']]
                # Adjust with cropped area
                row['x_center'] -= x_min
                row['y_center'] -= y_min
                # Normalize with cropped area
                row['x_center'] /= (x_max - x_min)
                row['y_center'] /= (y_max - y_min)
                row['width']    /= (x_max - x_min)
                row['height']   /= (y_max - y_min)
                row = row.values.astype('str')
                for box in range(len(row)):
                    text = ' '.join(row[box])
                    f.write(text)
                    f.write('\n')
        
    os.makedirs(f'{output_path}labels/train/{fold}/', exist_ok=True)
    os.makedirs(f'{output_path}images/train/{fold}/', exist_ok=True)
    os.makedirs(f'{output_path}labels/val/{fold}/',   exist_ok=True)
    os.makedirs(f'{output_path}images/val/{fold}/',   exist_ok=True)
    
    train_df = df[df[f'fold{fold}']==1].reset_index(drop=True)
    val_df   = df[df[f'fold{fold}']==0].reset_index(drop=True)
    make(train_df, "train")
    make(val_df,   "val")

In [8]:
class_names = list(df_train.label.unique())

def make_training_yaml(fold):
    with open(f"./yolov5_{fold}.yaml", "w+") as yf:
        yaml.dump({
            "train": f"{CFG['output_path']}images/train/{fold}/",
            "val"  : f"{CFG['output_path']}images/val/{fold}/",
            "nc"   : df_train.label.nunique(),
            "names": class_names
        }, yf, default_flow_style=False)

In [9]:
!cp ./yolov5/models/{CFG['model']}.yaml ./{CFG['model']}.yaml

with open(f"./{CFG['model']}.yaml", "r+") as yf:
    opened_yf = yaml.load(yf)
    !rm -rf ./{CFG['model']}.yaml
    opened_yf["nc"] = 1
    with open(f"./{CFG['model']}.yaml", "w") as yf2:
        yaml.dump(opened_yf, yf2, default_flow_style=False)

In [10]:
!cat ./yolov5/data/hyps/hyp.scratch.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (effic

### Run YOLO training

In [11]:
!rm -rf ./nfl
!rm -rf ./yolov5/runs
!rm -rf ./runs*

for i in range(CFG["fold_num"]):
    print("Making a yaml file for training and validation data...")
    create_file(df_train, i, CFG['input_path'], CFG['output_path'])
    print("Making a yaml file for configuration of YOLOv5...")
    make_training_yaml(i)

    print("Running YOLOv5 training...")
    !python ./yolov5/train.py \
        --img     {CFG["size"]} \
        --weights {CFG["model"]}.pt \
        --batch   {CFG["batch_size"]} \
        --epochs  {CFG["epochs"]} \
        --hyp     ./yolov5/data/hyps/hyp.scratch.yaml \
        --data    ./yolov5_{i}.yaml \
        --cfg     ./{CFG["model"]}.yaml
    
    !rm -rf ./nfl/images/train/{i}
    !rm -rf ./wandb/
    !mv ./yolov5/runs ./runs_{i}
    print("Done")
    
    if CFG["fold_break"] == i+1:
        print("Met break threthold")
        break

  0%|          | 0/7459 [00:00<?, ?it/s]

Making a yaml file for training and validation data...


100%|██████████| 2488/2488 [01:45<00:00, 23.59it/s]


Making a yaml file for configuration of YOLOv5...
Running YOLOv5 training...
train: weights=yolov5m.pt, cfg=./yolov5m.yaml, data=./yolov5_0.yaml, hyp=./yolov5/data/hyps/hyp.scratch.yaml, epochs=8, batch_size=24, imgsz=736, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
2021-10-29 14:33:33.790763: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-29 14:33:38.310464: I tensorflow/stream_executor/platform/default/dso_loader.

  0%|          | 0/7461 [00:00<?, ?it/s]

Done
Making a yaml file for training and validation data...


100%|██████████| 2486/2486 [01:36<00:00, 25.65it/s]


Making a yaml file for configuration of YOLOv5...
Running YOLOv5 training...
train: weights=yolov5m.pt, cfg=./yolov5m.yaml, data=./yolov5_1.yaml, hyp=./yolov5/data/hyps/hyp.scratch.yaml, epochs=8, batch_size=24, imgsz=736, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
2021-10-29 17:31:34.468315: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-29 17:31:38.208748: I tensorflow/stream_executor/platform/default/dso_loader.

In [12]:
for i in range(CFG["fold_num"]):
    !mkdir result_{i}
    !cp ./runs_{i}/train/exp/results*  ./result_{i}/
    !cp ./runs_{i}/train/exp/weights/* ./result_{i}/
    if CFG["fold_break"] == i+1:
        break

In [13]:
!rm -rf yolov5_* {CFG["model"]}*
!rm -rf ./yolov5
!rm -rf ./nfl/
!rm -rf ./runs_*